## Introduction to LZ77 Compression

**LZ77** is a lossless data compression algorithm developed by Abraham Lempel and Jacob Ziv in 1977.

The key idea is to **replace repeated sequences** in a string with references to previous occurrences. It uses a **sliding window** mechanism to find these repetitions.

**LZ77 Outputs :** Each compressed piece of data is a *triple* (offset, length, next_character):

- **offset** : how far back to look in the search buffer
- **length** : number of characters to copy
- **next_character** : the next new character following the match

**Sliding Window Structure :** LZ77 uses two buffers:

- **Search buffer**: a window of recently seen text (past)
- **Look-ahead buffer**: text currently being matched (future)


As the algorithm proceeds, the window **slides forward**, trying to match the longest substring in the search buffer

**Summary of Workflow :**

1. At each position in the text, search backward for the longest match.
2. If a match is found, encode it as (offset, length, next character)
3. If no match is found, encode as (0, 0, current character)
4. Slide the window forward and repeat.

**LZ77 Compression – Pseudocode :**

Here is the simplified pseudocode of the LZ77 algorithm using a sliding window:

<p align="center">
  <img src="img/LZ77/lz77.png" width="800"/>
  <br>
  <em>LZ77 Peudocode Wiki</em>
</p>


### Example
Text : `'abababab'`

Let’s say:

- Search buffer = 6 characters
- Look-ahead buffer = 4 characters

**Step 1:** start with:

- Search:   [empty]
- Look-ahead: a b a b

Since nothing in the search buffer, LZ77 outputs: `(0, 0, 'a')`

Move forward one character.

**Step 2:**

- Search:   a
- Look-ahead: b a b

Nothing matches 'b', so: `(0, 0, 'b')` 

Now move forward one character.

**Step 3:**

- Search:   a b
- Look-ahead: a b

Now we see "a b" in the search buffer!

We output: `(offset=2, length=2, next char='a')` 

Because from 2 characters ago (the 'a'), we can repeat 2 characters ("a b"), and the next character is 'a'.

We continue...

**Summary of Output:**
For the string a b a b a b a b, we might get something like:

(0,0,'a')  
(0,0,'b')  
(2,2,'a')  
(2,2,'')    <- optional if at end

### LZ77 Compression Function in Python

In [52]:
from typing import List, Tuple

# LZ77 Compression Algorithm
def lz77_compress(text: str, window_size: int = 20, lookahead_buffer_size: int = 15) -> List[Tuple[int, int, str]]:
    """
    Compress a string using the LZ77 algorithm

    :param text: The input string to compress
    :param window_size: The size of the sliding window
    :param lookahead_buffer_size: The size of the lookahead buffer

    :return: A list of tuples, each containing (distance, length, next character)
    """
    i = 0
    output = []

    while i < len(text):
        max_match_distance = 0
        max_match_length = 0
        next_char = ''
        start_index = max(0, i - window_size)
        window = text[start_index:i]

        # Find longest match in the window
        for j in range(1, min(lookahead_buffer_size, len(text) - i) + 1):
            substring = text[i:i + j]
            pos = window.rfind(substring)
            if pos != -1:
                max_match_distance = i - (start_index + pos)
                max_match_length = j
                if i + j < len(text):
                    next_char = text[i + j]
                else:
                    next_char = ''
            else:
                break

        if max_match_length > 0:
            output.append((max_match_distance, max_match_length, next_char))
            i += max_match_length + 1
        else:
            output.append((0, 0, text[i]))
            i += 1

    return output

In [ ]:
compressed_output = lz77_compress("banana")
compressed_output

[(0, 0, 'b'), (0, 0, 'a'), (0, 0, 'n'), (2, 2, 'a')]

### LZ77 Decompression Function


In [54]:
def lz77_decompress(compressed_output):
    output = []

    for offset, length, char in compressed_output:
        if offset == 0 and length == 0:
            output.append(char)
        else:
            start = len(output) - offset
            for i in range(length):
                output.append(output[start + i])
            if char:
                output.append(char)

    return ''.join(output)

In [55]:
# Test the decompression
original = lz77_decompress(compressed_output)
original

'banana'

## Introduction to LZ78 

**LZ78** is a lossless data compression algorithm introduced by Abraham Lempel and Jacob Ziv in 1978.

Unlike LZ77, which uses a sliding window, LZ78 builds a dictionary of phrases (substrings) as it reads the input.

**Key Concept**

- Read the input character by character.
- Build and maintain a **dictionary of known phrases**.
- Replace new occurrences with a reference to the dictionary instead of repeating the whole phrase.

**Output Format**

LZ78 outputs a sequence of **pairs** (index, character):


Where:
- *index* = index of a previously seen phrase (or 0 if it's a new one)
- *character* = the next character that extends the phrase

The pair is then added as a **new entry in the dictionary** : dictionary[index] + character

This allows repeated phrases to be stored efficiently.


**LZ78 Compression – Pseudocode**

Below is the pseudocode describing how the LZ78 algorithm works:


<p align="center">
  <img src="img/LZ77/lz78.png" width="800"/>
  <br>
</p>


### Example 

String : `wawawawa`

We'll initialize:

- dictionary = {} (empty at start)
- Phrase index starts from 1

We read characters and build phrases like this:

**Step 1:** 

Input: `"w"`   
No match in dictionary → emit: `(0, 'w')`  

Add "w" to dictionary with index 1: `1: "w"`  

**Step 2:**

Next character: `"a"`   
Not in dictionary → emit: `(0, 'a')`   

Add "a" to dictionary: `2: "a"`   

**Step 3:**

Next characters: `"w"` → `"wa"`   
`"w"` is in dictionary (index 1)  
Now we look ahead: "wa" is new → emit: `(1, 'a')` ← `(prefix "w", then "a")`

Add "wa" to dictionary: `3: "wa"`

**Step 4:**

Next: `"w"` → `"wa"` → `"waw"`  
`"wa"` is in dictionary (index 3)    
Next is "waw" — new → emit: `(3, 'w')`

Add "waw" to dictionary: `4: "waw"`

And so on...


Example: `"banana"`

Let’s compress "banana" with LZ78:

1. b → not found → `(0, 'b')` → add `"b"` as 1
2. a → not found → `(0, 'a')` → add `"a"` as 2
3. n → not found → `(0, 'n')` → add `"n"` as 3
4. a → found at 2, next is n → `"an"` not in dictionary → `(2, 'n')` → add `"an"` as 4
5. a → found at 2, next is end → `(2, '')` or nothing to add.

### LZ78 Compression Function


In [1]:
def lz78_compress(text):
    dictionary = {}
    data = []
    phrase = ""
    dict_size = 1  # dictionary indices start from 1

    for char in text:
        if phrase + char in dictionary:
            phrase += char
        else:
            if phrase == "":
                data.append((0, char))
            else:
                data.append((dictionary[phrase], char))
            dictionary[phrase + char] = dict_size
            dict_size += 1
            phrase = ""

    if phrase != "":
        data.append((dictionary[phrase], ""))

    return data


In [6]:
compressed_output = lz78_compress("banana")
compressed_output


[(0, 'b'), (0, 'a'), (0, 'n'), (2, 'n'), (2, '')]

#### LZ78 Decompression Function

In [7]:

def lz78_decompress(compressed_data):
    dictionary = {0: ""}  # index 0 corresponds to an empty string
    output = []
    dict_size = 1

    for index, char in compressed_data:
        entry = dictionary.get(index, "") + char
        output.append(entry)
        dictionary[dict_size] = entry
        dict_size += 1

    return ''.join(output)

# Test decompression
decompressed = lz78_decompress(compressed_output)
print("Decompressed string:", decompressed)


Decompressed string: banana


## LZW Compression

**LZW** is a lossless compression algorithm introduced by Terry Welch in 1984.  

It is an improvement of LZ78.

The big idea:

- Build a dictionary of phrases, like LZ78.  
- But instead of sending `(index, character)` pairs like LZ78, LZW only sends the `index` of the longest matching phrase.

This means it’s even more compact than LZ78.

**Step-by-Step How LZW Works**

Initial Setup:

Start with a dictionary of all single characters (like all ASCII characters).  
Each character has a unique index.  
We read the input left to right, building up longer phrases.  
Output the index of the longest match seen so far.  
Then add a new phrase to the dictionary: `previous match + next character`.

**Example**

String : `"ABABABA"`

**Step 0:** Initial Dictionary

| Index | Char |
|-------|--------|
|  0    | A      |
|  1    | B      |


We’ll add new phrases starting at index 2.

**Step 1:**

Current = `A`  
Look ahead = `AB`  

A is in the dictionary → match  
AB is not in the dictionary  

→ Output index of `A = 0`  
→ Add `"AB"` to dictionary at `index 2`  


**Step 2:**

Current = `B`  
Look ahead = `BA`  

B is in the dictionary → match
BA is not in the dictionary

→ Output index of `B = 1`
→ Add `"BA"` to dictionary at `index 3`

**Step 3:**

Current = `A` 
Look ahead = `AB` 

AB is now in the dictionary at `index 2`
→ Keep going

Look ahead = `ABA` → not in dictionary

→ Output index of `"AB" = 2`
→ Add `"ABA"` to dictionary at `index 4`

**Step 4:**

Current = `A`
Look ahead = `AB` → still in dictionary
Look ahead = `ABA` → in dictionary `(index 4)`
→ Continue reading but end of string is reached

→ Output index of `"ABA" = 4`

Final Output: `[0, 1, 2, 4]`

And the dictionary was built as:

| Index | Phrase |
|-------|--------|
|  0    | A      |
|  1    | B      |
|  2    | AB     |
|  3    | BA     |
|  4    | ABA    |




### LZW Compression Function

In [ ]:
def lzw_compress(text):
    # Initialize dictionary with single characters
    dictionary = {chr(i): i for i in range(256)}  # ASCII characters
    next_code = 256
    phrase = ""
    output = []

    for char in text:
        if phrase + char in dictionary:
            phrase += char
        else:
            output.append(dictionary[phrase])
            dictionary[phrase + char] = next_code
            next_code += 1
            phrase = char

    if phrase:
        output.append(dictionary[phrase])

    return output, dictionary


In [59]:
# Run LZW on "ABABABA"
text = "ABABABA"
compressed, final_dict = lzw_compress(text)

print("Compressed output:", compressed)
print("\nFinal dictionary entries (after 255):")
for k, v in final_dict.items():
    if v >= 256:
        print(f"{v}: '{k}'")


Compressed output: [65, 66, 256, 258]

Final dictionary entries (after 255):
256: 'AB'
257: 'BA'
258: 'ABA'


## LZ77 compression with time-series data

In [1]:
def lz77_compress_series(data, window_size=20, lookahead_buffer_size=15):
    i = 0
    compressed_data = []

    while i < len(data):
        match = (-1, -1)  # (offset, length)
        for j in range(max(0, i - window_size), i):
            length = 0
            while (length < lookahead_buffer_size and
                   i + length < len(data) and
                   data[j + length] == data[i + length]):
                length += 1
                if j + length >= i:
                    break
            if length > match[1]:
                match = (i - j, length)

        if match[1] > 0:
            next_elem = data[i + match[1]] if i + match[1] < len(data) else None
            compressed_data.append((match[0], match[1], next_elem))
            i += match[1] + 1
        else:
            compressed_data.append((0, 0, data[i]))
            i += 1

    return compressed_data


In [2]:
def lz77_decompress_series(compressed_data):
    decompressed = []

    for offset, length, elem in compressed_data:
        if offset == 0 and length == 0:
            decompressed.append(elem)
        else:
            start = len(decompressed) - offset
            for _ in range(length):
                decompressed.append(decompressed[start])
                start += 1
            if elem is not None:
                decompressed.append(elem)
    return decompressed


In [12]:
if __name__ == "__main__":
    # Sample time series (toy example)
    series = [1, 2, 3, 1, 2, 3, 4, 1, 2, 3, 4, 5, 2, 3, 4, 1, 2,3]
    print("Original Time Series:", series)

    compressed = lz77_compress_series(series)
    print("\nCompressed Output:")
    for item in compressed:
        print(item)

    decompressed = lz77_decompress_series(compressed)
    print("\nDecompressed Time Series:", decompressed)


Original Time Series: [1, 2, 3, 1, 2, 3, 4, 1, 2, 3, 4, 5, 2, 3, 4, 1, 2, 3]

Compressed Output:
(0, 0, 1)
(0, 0, 2)
(0, 0, 3)
(3, 3, 4)
(4, 4, 5)
(8, 6, None)

Decompressed Time Series: [1, 2, 3, 1, 2, 3, 4, 1, 2, 3, 4, 5, 2, 3, 4, 1, 2, 3]


In [ ]:
import sys

def calculate_compression_ratio(original_data, compressed_data):
    # Estimate size in bytes: each int = 4 bytes, float = 8 bytes (approx)
    original_size = sys.getsizeof(original_data) + sum(sys.getsizeof(x) for x in original_data)
    compressed_size = sys.getsizeof(compressed_data) + sum(sys.getsizeof(t) for t in compressed_data)
    ratio = original_size / compressed_size
    rate = 1 - (compressed_size / original_size)
    return ratio, rate


In [14]:
rat, rate = calculate_compression_ratio(series, compressed)

print(f"\nCompression Ratio: {rat:.2f}, Compression Rate: {rate:.2%}")


Compression Ratio: 1.40, Compression Rate: 28.41%
